this notebook best viewed here: https://nbviewer.jupyter.org/github.com/brandonlind/BURT/002_trim_and_map.ipynb

In [1]:
from pythonimports import *

In [2]:
DIR = op.realpath('/home/lindb/eckertlab/BURT')
DIR

'/gpfs_fs/home/eckertlab/BURT'

In [3]:
tandmdir = makedir(op.join(DIR, '002_trim_and_map'))

# trim and map to de novo assembly

- dDocent was used to create an de novo assembly from trimmed reads
- dDocent documentation refers to trimmomatic, but uses fastp. So I'll re-trim and map. I can also handle read groups during mapping this way, dDocent does not account for different read groups across samples.

#### create datatable.txt

- datatable.txt is used as input to start the varscan_pipeline - github.com/brandonlind/varscan_pipeline
- I am using the first few steps of this pipeline to trim and map __only__
- I've updated the pipeline to handle SGE scheduler (it was slurm-only before)

In [4]:
fqs = pd.read_table('/gpfs_fs/home/eckertlab/BURT/seq/burt_fastq_data.txt')
fqs.head()

,fastq,sample_name,species,state,ind,popn,spp_popn,run,lib,seq_center,md5,RGPL,RGSM,RGLB,RGID,flowcell_lane,RGPU,num_reads
0,/home/cfriedline/eckertlab/projects/burt/seq/1...,T-AR-2-7,T,AR,7,T-AR-2,AR-2,160520,Burt0,NARF,1ae81d40ba5b1497e7ba9f5733c494b4,ILLUMINA,T-AR-2-7,Burt0,T-AR-2-7.160520.Burt0,C7WRGACXX.1,C7WRGACXX.1.T-AR-2-7,1277586
1,/home/cfriedline/eckertlab/projects/burt/seq/1...,T-TX-1-19,T,TX,19,T-TX-1,TX-1,160520,Burt0,NARF,eb991e8800c31b632dfc0f4279e9e44a,ILLUMINA,T-TX-1-19,Burt0,T-TX-1-19.160520.Burt0,C7WRGACXX.1,C7WRGACXX.1.T-TX-1-19,2798860
2,/home/cfriedline/eckertlab/projects/burt/seq/1...,G-SC-1-3,G,SC,3,G-SC-1,SC-1,160520,Burt0,NARF,e5cf36460b50c6467a49f4b157bb16c1,ILLUMINA,G-SC-1-3,Burt0,G-SC-1-3.160520.Burt0,C7WRGACXX.1,C7WRGACXX.1.G-SC-1-3,2312577
3,/home/cfriedline/eckertlab/projects/burt/seq/1...,P-LA-2-10,P,LA,10,P-LA-2,LA-2,160520,Burt0,NARF,a1fab3ce615d708f53eb836f13fbba7c,ILLUMINA,P-LA-2-10,Burt0,P-LA-2-10.160520.Burt0,C7WRGACXX.1,C7WRGACXX.1.P-LA-2-10,1583884
4,/home/cfriedline/eckertlab/projects/burt/seq/1...,T-AL-1-2,T,AL,2,T-AL-1,AL-1,160520,Burt0,NARF,16efa4d02874e6f5cf747e40bdbfb0dd,ILLUMINA,T-AL-1-2,Burt0,T-AL-1-2.160520.Burt0,C7WRGACXX.1,C7WRGACXX.1.T-AL-1-2,1321261


In [10]:
# get unique basenames names for all fastqs
bcounts = Counter()
bnames = {}
for fastq in fqs['fastq']:
    bname = op.basename(fastq).replace(".R1.fastq.gz", "_R1.fastq.gz")
    bcounts[bname] += 1
    if not bname in list(bnames.values()):
        bnames[fastq] = bname
    else:
        bname = bname.replace(".fastq.gz", "_%s.fastq.gz" % bcounts[bname])
    bnames[fastq] = bname
len(bnames), luni(bnames.values())

(1709, 1709)

In [12]:
bnames[fastq]

'T-LA-3-12_R1.fastq.gz'

In [13]:
# read in datatable.txt template, create blank template
dt = pd.read_table('/home/lindb/g/varscan_pipeline/datatable.txt')
dt = pd.DataFrame(columns=dt.columns)
display(dt)  # display blank df to show col names

for line in nb(fqs.index):
    row = nrow(dt)
    dt.loc[row, 'sample_name'] = fqs.loc[line, 'sample_name']
    dt.loc[row, 'library_name'] = fqs.loc[line, 'lib']
    dt.loc[row, 'file_name_r1'] = bnames[fqs.loc[line, 'fastq']]
    dt.loc[row, 'rgid'] = fqs.loc[line, 'RGID']
    dt.loc[row, 'rglb'] = fqs.loc[line, 'RGLB']
    dt.loc[row, 'rgpu'] = fqs.loc[line, 'RGPU']
    dt.loc[row, 'rgsm'] = dt.loc[row, 'sample_name']
dt['pool_name'] = 'BURT'
dt['ploidy'] = 2
dt['file_name_r2'] = np.nan
dt['adaptor_1'] = np.nan
dt['adaptor_2'] = np.nan
dt['ref'] = '/home/lindb/eckertlab/BURT/dDocent_denovo_assembly/reference_pseudo.fasta'
dt['rgpl'] = 'ILLUMINA'
dt.head()

,sample_name,library_name,pool_name,ploidy,file_name_r1,file_name_r2,adaptor_1,adaptor_2,ref,rgid,rglb,rgpl,rgpu,rgsm


100%|██████████| 1709/1709 [00:22<00:00, 77.33it/s]


,sample_name,library_name,pool_name,ploidy,file_name_r1,file_name_r2,adaptor_1,adaptor_2,ref,rgid,rglb,rgpl,rgpu,rgsm
0,T-AR-2-7,Burt0,BURT,2,T-AR-2-7_R1.fastq.gz,NaN,NaN,NaN,/home/lindb/eckertlab/BURT/dDocent_denovo_asse...,T-AR-2-7.160520.Burt0,Burt0,ILLUMINA,C7WRGACXX.1.T-AR-2-7,T-AR-2-7
1,T-TX-1-19,Burt0,BURT,2,T-TX-1-19_R1.fastq.gz,NaN,NaN,NaN,/home/lindb/eckertlab/BURT/dDocent_denovo_asse...,T-TX-1-19.160520.Burt0,Burt0,ILLUMINA,C7WRGACXX.1.T-TX-1-19,T-TX-1-19
2,G-SC-1-3,Burt0,BURT,2,G-SC-1-3_R1.fastq.gz,NaN,NaN,NaN,/home/lindb/eckertlab/BURT/dDocent_denovo_asse...,G-SC-1-3.160520.Burt0,Burt0,ILLUMINA,C7WRGACXX.1.G-SC-1-3,G-SC-1-3
3,P-LA-2-10,Burt0,BURT,2,P-LA-2-10_R1.fastq.gz,NaN,NaN,NaN,/home/lindb/eckertlab/BURT/dDocent_denovo_asse...,P-LA-2-10.160520.Burt0,Burt0,ILLUMINA,C7WRGACXX.1.P-LA-2-10,P-LA-2-10
4,T-AL-1-2,Burt0,BURT,2,T-AL-1-2_R1.fastq.gz,NaN,NaN,NaN,/home/lindb/eckertlab/BURT/dDocent_denovo_asse...,T-AL-1-2.160520.Burt0,Burt0,ILLUMINA,C7WRGACXX.1.T-AL-1-2,T-AL-1-2


In [14]:
dt.to_csv(op.join(tandmdir, 'datatable.txt'), sep='\t', index=False)

In [15]:
# make symlinks to parentdir
for src in nb(fqs['fastq']):
    dst = op.join(tandmdir, bnames[src])
    os.symlink(src, dst)

100%|██████████| 1709/1709 [00:01<00:00, 1591.38it/s]
